In [16]:
import tempfile as tf
import re
import shutil
import os

In [17]:
from bib2md import md2tex

In [18]:
tf.tempdir = '.'
build_dir_h = tf.TemporaryDirectory(prefix='b2md_')
build_dir = build_dir_h.name
tf.tempdir = build_dir

In [19]:
infile = 'test_mdcite.md'
bibliography = ['mypub.bib']
infile = open(infile,'r')

In [20]:
def is_local(path):
    '''Decides whether given argument is installed or local.
    
    Returns a pair (local, last) where local is the boolean
    answer and last is the last part of the path (e.g filename).'''
    local = False
    if path.startswith('.') or path.startswith('/'):
        local = True
    last = path.split('/')[-1]
    return local, last

In [49]:
def extract_cite_cmds(infile):
    '''searches `infile` for TeX cite commands and returns
    set of triples `(full_cmd, cite type, cite key)`
    '''
    infile.seek(0)
    content = infile.read()
    # locate each citation record and store them in a set of tuples:
    # (full string, cite command, key)
    p = re.compile(r'(\\([a-z]*cite)\{([^}]+)\})')
    m = p.findall(content)
    return set(m)

def generete_tex(cites, build_dir, bibliography,
                 print_biblio=True, bib_style='md',
                 template='bib2md.latex',verbose=False):
    '''Generate texfiles for conversion to md for each cite.
    
    For each cite in `cites`, generate the corresponding TeX file
    `build_dir/type.key.tex` that contains the full_cmd of the cite
    using the specified Pandoc `template`.
    
    Parameters
    ==========
    `bibliography` : list of Strings
        filenames with bibliography entries to be used
        in the further processing.
    `bib_style`: String
        name of biblatex style
    '''
    if build_dir is None:
        build_dir = '.'
    # create files for each different citation record
    for cmd, cite, key in cites:
        inname = '{}/{}.{}.source.md'.format(build_dir,cite,key)
        texname = '{}/{}.{}.tex'.format(build_dir,cite,key)
        result = '{}/{}.{}.md'.format(build_dir,cite,key)
        with open(inname,'w') as md_f:
            print(cmd,file=md_f)
        md2tex(inname,bibliography,
               out_file=texname,print_biblio=False)
        
def prepare_build_dir(build_dir, bibliography, cites, bib_style='md', 
                      template='bib2md.latex', 
                      htlatex_cfg='./md.cfg'): ## TODO Update this to install location!
    '''Copies the necessary files into `build_dir`.'''
    for b in bibliography:
        shutil.copy(b, build_dir)
    if is_local(bib_style)[0]:
        shutil.copy(bib_style+'.bbx',build_dir)
        shutil.copy(bib_style+'.cbx',build_dir)
        if os.path.exists(bib_style+'.dbx'):
            shutil.copy(bib_style+'.dbx',build_dir)
    for f in [template, htlatex_cfg]:
        if is_local(f):
            shutil.copy(f, build_dir)
    shutil.copy('Makefile', build_dir)# TODO Generate Makefile using actual names
    
    ### TODO: Make a function
    make_template = '''default: all

%.pdf : %.tex {biblio_str} {bib_style}.*bx
	xelatex $(patsubst %.pdf,%.tex,$@) 
	biber $(patsubst %.pdf,%,$@) 
	xelatex $(patsubst %.pdf,%.tex,$@)

%.html : %.pdf {htlatex_cfg}
	htxelatex $(patsubst %.html,%.tex,$@) {htlatex_cfg} " -cunihtf -utf8"

### Removes silly link anchor produced by tex4ht for the firs entry
%.md : %.html
	pandoc -f html+tex_math_dollars -t markdown $< | sed 's/\[\]{{#page.1}}\[\]{{#X0-}}//' > $@

clean_mess:
	latexmk -CA
	rm -f *.4* *.bbl *.css *.html *.idv *.lg *.run.xml *.tmp *.xref *.xdv

all: {md_files}
'''
    makefile = make_template.format(
        biblio_str=' '.join(pure_biblio),
        bib_style=is_local(bib_style)[1],
        htlatex_cfg=is_local(htlatex_cfg)[1],
        md_files=' '.join(['{}.{}.md'.format(cite,key) for _, cite, key in cites]))
    print(makefile)

In [50]:
bib_style = './md'
cites = extract_cite_cmds(infile)

pure_biblio = [is_local(b)[1] for b in bibliography]
generete_tex(cites, build_dir, 
             pure_biblio, bib_style=bib_style)
prepare_build_dir(build_dir, bibliography, cites, './md', 
                  './bib2md.latex', './md.cfg')

{('\\cite{babiak.15.cav}', 'cite', 'babiak.15.cav'), ('\\mdcite{blahoudek.17.lpar}', 'mdcite', 'blahoudek.17.lpar'), ('\\mdcite{babiak.15.cav}', 'mdcite', 'babiak.15.cav')}
default: all

%.pdf : %.tex mypub.bib md.*bx
	xelatex $(patsubst %.pdf,%.tex,$@) 
	biber $(patsubst %.pdf,%,$@) 
	xelatex $(patsubst %.pdf,%.tex,$@)

%.html : %.pdf md.cfg
	htxelatex $(patsubst %.html,%.tex,$@) md.cfg " -cunihtf -utf8"

### Removes silly link anchor produced by tex4ht for the firs entry
%.md : %.html
	pandoc -f html+tex_math_dollars -t markdown $< | sed 's/\[\]{#page.1}\[\]{#X0-}//' > $@

clean_mess:
	latexmk -CA
	rm -f *.4* *.bbl *.css *.html *.idv *.lg *.run.xml *.tmp *.xref *.xdv

all: cite.babiak.15.cav.md mdcite.blahoudek.17.lpar.md mdcite.babiak.15.cav.md



In [51]:
build_dir_h.cleanup()